Hi! This notebook is a labour of love. 
Please credit https://github.com/MargheritaPhilipp should you ever pass it on.

### 0. Intro

A bit on markdown and native python things.

This section is written in mark down. Here you can 
1. Create numbered lists, make words **bold**, *italic*, or ***both***,
- create bullet points lists
and highlight `variable_names`

Below are some example of things you can do that are just native to python and don't require the import of any libraries into your notebook.

Also note that normally you would import all the libraries you need at the start of a .py or of your notebook. Here we make an exception to make it clear what sections use which of the libraries (at least for the first time within the notebook).


In [ ]:
# adding numbers
print(type(4.2))
print(4.2 + 3.8 )

# "adding" strings together
print(type('hello'))
print('hello' + "_my_" + 'friend')

# "adding" lists together
print(type(['do', 'I']))
print(['do', 'I']+ ['know', "you"])

### 1. EDA on WB pop data

Each section links to some of the questions that were tasks in class.

#### 1.1 Importing data

Relative paths are better for collaboration.


Addresses the following questions from class:
- Import the WB data (can you do it via a relative path?)

In [394]:
import pandas as pd

# importing with absolute path: you won't be able to run this directly 

df = pd.read_csv('/Users/margheritaphilipp/Documents/GitHub/brushup/data/WB_pop_clean.csv')

In [ ]:
import os

# importing with relative path: if you have saved this file in a folder that has a data subfolder with the same csv inside, you can run this directly 

# get current working directory
cwd = os.getcwd()
print(cwd)

df_og = pd.read_csv(cwd + '/data/WB_pop_clean.csv')

#### 1.2. Starting inspection



Addresses the following questions from class:
- Display the head, check for missing values
- Find the min and max values - overall and just for 2023
- Which countries do they belong to?
- Inspect the values in the “Country Code” column

In [ ]:
# show the dimensions (rows and columns) of the data set and display first few rows
print(df.shape)

# other options:
# df.tail(2)
# df.sample(4)

df_og.head() # default is 5

In [ ]:
# sometimes not all columns are visible so it can be useful to get the full list

df_og.columns

Note that while .head() is a ***method*** I apply to the data frame, .shape and .columns are ***attributes*** of the data frame object/ class that I can call

- Example with planets class (10 mins): https://www.youtube.com/watch?v=LwFnF9XoEf
- Example with house class (7 mins): https://www.youtube.com/watch?v=3zoyA3U2Ka0


In [ ]:
# the info method also tells us which columns are present and what data type they contain
# we know from the shape attribute that there are 218 rows and it seems that all rows contain data (are non-null), i.e. we don't have missing values

df_og.info()

In [ ]:
# statistical summary of the numerical columns - we can already see a suspiciously high maximum value...

df_og.describe()

In [ ]:
# if I just want to find the min and max values for a speficic column:

print('mix and max vals for 2023: ', df['2023'].min(), df['2023'].max())

In [ ]:
# one way to get the whole row for these values is to use loc

df_og.loc[df['2023'] == df['2023'].min()]

In [ ]:
# but this method is a bit more elegant and flexible

df_og.nlargest(2, '2023')

In [ ]:
df_og.nsmallest(5, ['2023', '2001'])

In [ ]:
# inspecting the country column: note that the lentgh of the value counts is 218, same as the number of unique values, so each country only appears once

print(df_og['Country Name'].nunique()) # same as len(df['Country Name'].unique())

print(df_og['Country Name'].unique())

df_og['Country Name'].value_counts() 

#### 1.3. Basic manipulations and outliers 

##### Distribution and outliers

- Are there any outliers?
    - Show them in a box plot.
    - What could you do with them?
- Can you plot a histogram and/ or kernel density for the values in 2023?

In [405]:
# it's good practice to make a copy of the data set before you start manipulating it
# that way you also don't have to import the data from scratch each time you made an error (e.g. removing a column you end up wanting to keep)

df = df_og.copy()

In [406]:
#!pip3 install matplotlib # you may need to install matplotlib - which you can do directly in this notebook with the !pip3 :)

In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt  

df.boxplot(column =['2003', '2013', '2023'], grid = False) # this is native to pandas which uses matplotlib without us having to call it
plt.show()


In [ ]:
# we know the "world" row is the outlier, which is not a country, so let's remove it
# remember: we can just re-run the cell that copies the og data to undo this change)

# option 1: only keep other rows
df = df[df['Country Name'] != 'World']

# option 2: drop specific row
#df = df.drop(df.loc[df['Country Name'] == 'World'])

# option 3: see the option with ~ below (just search for the tilde with ctr+f)

# check that there is one row less:
print(df.shape)


In [ ]:
import seaborn as sns  # remember to install this in your environment first

# plot again, this time with seaborn, and you can watch India overtake China

sns.set_style("whitegrid") 

columns_to_plot = ['2003', '2013', '2023']
  
sns.boxplot(data = df[columns_to_plot]) 
plt.show()

In [ ]:
# play with the bins variable of the histogram
df.hist(column=['2023'], bins=20)
plt.show()

In [ ]:
import scipy as sp

# kernel densities also give an idea of the overall distribution

df['2023'].plot.density()
plt.show()

##### Population growth over time


- Which county has seen the greatest population growth from the start to the end of the timeline?
    - In absolute terms?
    - In relative terms?
    - What about the greatest decrease?

In [ ]:
# create a new column that shows the difference between 2001 and 2023 - not that the absolute value would not be helpful

df['gwth_2023_2001_abs'] = df['2023'] - df['2001'] 
df['gwth_2023_2001_rel'] = round(df['gwth_2023_2001_abs'] / df['2001'], 2)
df.head(2)

In [ ]:
# find the highest growth values - you can of course to the equivalent for the smallest (i.e. most negative) values
# bonus: can you find the countries with the samlles positive growth?

print('Greatest absolute growth from 2001 to 2023: ')
print('   - Country: ', df.nlargest(1, 'gwth_2023_2001_abs')['Country Name'])
print('   - Growth: ', df.nlargest(1, 'gwth_2023_2001_abs')['gwth_2023_2001_abs'])

print('\n') # this prints a line break

print('Greatest relative growth from 2001 to 2023: ')
print('   - Country: ', df.nlargest(1, 'gwth_2023_2001_rel')['Country Name'])
print('   - Growth: ', df.nlargest(1, 'gwth_2023_2001_rel')['gwth_2023_2001_rel'])

##### Comparing rows (countries)


Addresses the following questions from class:
- Show just the rows for Spain and your country/ countries of origin
    - Can you create a new row that shows the difference in population over time?
    - Can you plot this?

In [ ]:
# displaying only selected countries

df.loc[( df['Country Name']== 'Germany') | (df['Country Name']== 'United Kingdom') | (df['Country Name']== 'Spain')]

In [ ]:
# slightly neater and more flexible

country_choices = ['Germany', 'United Kingdom', 'Spain']

df.loc[df['Country Name'].isin(country_choices)].sort_values('2023', ascending=False) # here I am also specifying what to sort by

In [ ]:
import numpy as np 

# to take the difference row-wise, I need to select only the columns that contain numbers and are relevant

year_cols = ['2001', '2002', '2003', '2011', '2012', '2013', '2021', '2022', '2023']

df = df.copy() # otherwise you'll get a warning about assinging values that might overwrite existing ones

# we can also make the index (i.e. the row name) more meaningful by inserting the country code
df.set_index(df['Country Code'], inplace=True)

# create rows if they don't exist
if 'diff_deu_gbr' not in df.index:
    df.loc['diff_deu_gbr'] = np.nan # making the default value Nan

if 'diff_deu_esp' not in df_subset.index:
    df.loc['diff_deu_esp'] = 0      # making the default value 0

for col in year_cols:
    df.loc['diff_deu_gbr', col] = df.loc['DEU', col] - df.loc['GBR', col]
    df.loc['diff_deu_esp', col] = df.loc['DEU', col] - df.loc['ESP', col]

df.loc[['DEU', 'GBR', 'ESP'], ['Country Name', 'Series Name'] + year_cols ]


In [ ]:
# plotting from rows
rows_to_plot = ['diff_deu_gbr', 'diff_deu_esp']
data_to_plot = df.loc[rows_to_plot, numerical_cols]

# transpose the data to make the columns the x-axis (years) and the rows the data series
data_to_plot = data_to_plot.T

# plot the data
data_to_plot.plot(kind='line', marker='o', figsize=(10, 6))

# specify labels and title
plt.title('How much smaller UK and Spain are compared to Germany')
plt.xlabel('Year')
plt.ylabel('Values')
plt.legend(title='Country/Row')

# show the plot
plt.show()

##### Better solution - using transpose to compare columns instead

In [ ]:
# the above is really cumbersome, so it's much easier to transpose the data from the beginning
# to work only with the subset we can store it in a new data frame

# country_choices = ['Germany', 'United Kingdom', 'Spain'] # this was already set above

df_subset = df.loc[df['Country Name'].isin(country_choices) ]

print(df_subset.shape)
df_subset

In [ ]:
# this is what tranposing does - note the nice column names thanks to changing the index
df_transposed = df_subset.T
df_transposed

In [ ]:
# I can drop the repeated information to make it neaters

to_drop = ['Country Code', 'Series Name', 'Series Code', 'Country Name'] # without this the transpose creates repeated values
df_subset2 = df_subset.drop(columns=to_drop)

df_transposed = pd.DataFrame(df_subset2.T)
df_transposed

In [ ]:
# now finding the differences is easier
df_transposed['diff_deu_esp'] = df_transposed['DEU'] - df_transposed['ESP']
df_transposed['diff_deu_gbr'] = df_transposed['DEU'] - df_transposed['GBR']
df_transposed

### 2. Background cleaning

What I did to give you a slightly nicer data set - shows how to store data frames as csvs and how to rename columns using a dictionary

In [ ]:
df_WB = pd.read_csv(cwd + '/data/WB_full.csv')
#df_meta = pd.read_csv(cwd + '/data/WB_metadata.csv')

df_WB.head()

In [ ]:
# cols_to_clean = [
#     '2001 [YR2001]', '2002 [YR2002]', '2003 [YR2003]', '2011 [YR2011]',
#     '2012 [YR2012]', '2013 [YR2013]', '2021 [YR2021]', '2022 [YR2022]', '2023 [YR2023]']

# cols_to_clean might include other non-year columns
cols_to_clean = df_WB.columns.tolist()

# Creating a dictionary to map the year columns from '2001 [YR2001]' to just '2001'
rename_dict = {col: col.split()[0] for col in cols_to_clean if '[YR' in col}

# Using rename to only change the year columns and leave the rest intact
df_WB = df_WB.rename(columns=rename_dict)

df_WB.head(2)

In [ ]:
df_pop = df_WB.loc[df_WB['Series Code']=='SP.POP.TOTL']

country_groups = [
    "AFE", "AFW", "ARB", "CSS", "CEB", "EAR", "EAS", "EAP", "TEA", "EMU", 
    "ECS", "ECA", "TEC", "EUU", "FCS", "HPC", "HIC", "IBD", "IBT", "IDB", 
    "IDX", "IDA", "LTE", "LCN", "LAC", "TLA", "LDC", "LMY", "LIC", "LMC", 
    "MEA", "MNA", "TMN", "MIC", "NAC", "INX", "OED", "OSS", "PSS", "PST", 
    "PRE", "SST", "SAS", "TSA", "SSF", "SSA", "TSS", "UMC"]

world = ["WLD"]

filtered_df = df_pop[~df_pop['Country Code'].isin(country_groups)]

filtered_df.shape


In [426]:
# download
#filtered_df.to_csv(cwd + '/data/WB_pop_clean.csv', index=False)